In [ ]:
class EventStudy():
    import pandas as pd
    import datetime
    def __init__(self,file_path):
        try:
            pd.read_csv(file_path)
        except TypeError:
            return f'Wrong File Type.'
        self.file = pd.read_csv(file_path)
        
    def datetime(self):
        self.dataframe = pd.DataFrame(self.file)
        self.company = self.dataframe['Ticker']
        self.year = self.dataframe['Year']
        self.month = self.dataframe['Month']
        self.day = self.dataframe['Day']
        self.market = self.dataframe['Market']
        self.industry = self.dataframe['Industry']
        
    def eventdate(self):
        eventdate = []
        for i in range(0,len(self.dataframe)):
            eventdate.append(datetime.datetime(self.year[i],self.month[i],self.day[i]))
        self.dataframe['EventDate'] = eventdate

    def sample_length(self):
        
        pass
    
    def reg(i,m,year,month,day,tbefore,tafter):
    
    from datetime import timedelta
    import pandas_datareader.data as web
    import fix_yahoo_finance as yf
    import matplotlib.pyplot as plt
    import numpy as np
    import statsmodels.api as sm
    event_date = datetime.datetime(year,month,day)
    start_date = event_date - timedelta(days = tbefore)
    end_date = event_date + timedelta(days = tafter)
    df = web.DataReader([i, m],'yahoo', start_date,end_date)
    df = df['Close']
    rets = df.pct_change()
    %matplotlib inline
    plt.scatter(rets[i],rets[m])
    X = rets[i]    
    X = sm.add_constant(X)
    y = rets[m]
    model = sm.OLS(y,X,missing='drop')
    result = model.fit()
    print(result.summary())
    (a,b) = result.params
    fig, ax = plt.subplots(figsize=(12,10))
    ax.plot(y)
    ax.plot(result.fittedvalues)
    rets['u'] = rets[i] - rets[m]*b - a
    rets['cumu'] = rets['u'].cumsum()
    fig, ax = plt.subplots(figsize=(12,10))
    ax.plot(rets['cumu'])
    
    def visualization(self):
        pass